In [2]:
import torch
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, GenerationConfig
from peft import get_peft_model, LoraConfig, TaskType

In [6]:
LOCAL_MODEL_PATH = "../ai_models/gemma-3-270m"
DTYPE = torch.float32
MODEL_OPTION = {"use_safetensors": True}
ADAPTER_FLAG = False
ADAPTER_PATH = ""

In [4]:
device = None
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("✅ Using Apple Silicon GPU (MPS)")
else:
    device = torch.device("cpu")
    print("⚠️ MPS not available; using CPU")

✅ Using Apple Silicon GPU (MPS)


In [5]:
def get_tokenizer(model_path):
    print("🔄 Loading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(
        model_path,
        use_fast=True,
        padding_side="left",  # 배치 추론 대비 안전
        use_safetensors=True,
    )
    if tokenizer.pad_token is None:
        print("⚠️ pad_token이 없어서 eos_token으로 설정합니다.")
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.pad_token_id = tokenizer.eos_token_id

    tokenizer.padding_side = "left"
    return tokenizer

def get_model(model_path, dtype, option):
    print("🔄 Loading model...")
    return AutoModelForCausalLM.from_pretrained(
        model_path,
        dtype=dtype,
        low_cpu_mem_usage=True,
        use_safetensors=option["use_safetensors"],
    )

def set_model_to_device(model, device):
    print("🔄 Moving model to device...")
    model.to(device)
    model.eval()
    return model

In [25]:
# ==== 프롬프트 구성: chat 템플릿 자동 감지 ====
def build_inputs(tokenizer, user_text: str):
    # tokenizer가 chat 템플릿을 제공하면 그걸 사용 (instruct 모델에 유리)
    has_chat = (hasattr(tokenizer, "chat_template") and tokenizer.chat_template)
    if has_chat:
        messages = [{"role": "user", "content": user_text}]
        enc = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1024,
        )
    else:
        # 베이스 모델일 경우 단순 프롬프트
        enc = tokenizer(
            user_text,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1024,
        )
    return enc

In [8]:
tokenizer = get_tokenizer(LOCAL_MODEL_PATH)
model = get_model(LOCAL_MODEL_PATH, DTYPE, MODEL_OPTION)
model = set_model_to_device(model, device)

print("load done.")

🔄 Loading tokenizer...
🔄 Loading model...
🔄 Moving model to device...
load done.


In [60]:
# generate config
# # 옵션에 따른 추론 테스트
# 퍼포먼스 향상 테스트
gen_cfg = GenerationConfig(
    max_length=None,
    max_new_tokens=128,                 # 의미 없음
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

In [32]:
import json

print(json.dumps(gen_cfg.to_dict(), indent=4, ensure_ascii=False))

{
    "max_length": 20,
    "max_new_tokens": 128,
    "min_length": 0,
    "min_new_tokens": null,
    "early_stopping": true,
    "max_time": null,
    "stop_strings": null,
    "do_sample": false,
    "num_beams": 3,
    "num_beam_groups": 1,
    "use_cache": true,
    "cache_implementation": null,
    "cache_config": null,
    "return_legacy_cache": null,
    "prefill_chunk_size": null,
    "temperature": 1.0,
    "top_k": null,
    "top_p": null,
    "min_p": null,
    "typical_p": 1.0,
    "epsilon_cutoff": 0.0,
    "eta_cutoff": 0.0,
    "diversity_penalty": 0.0,
    "repetition_penalty": 1.2,
    "encoder_repetition_penalty": 1.0,
    "length_penalty": 1.0,
    "no_repeat_ngram_size": 3,
    "bad_words_ids": null,
    "force_words_ids": null,
    "renormalize_logits": false,
    "constraints": null,
    "forced_bos_token_id": null,
    "forced_eos_token_id": null,
    "remove_invalid_values": false,
    "exponential_decay_length_penalty": null,
    "suppress_tokens": null,
    

In [62]:
print("===== generate =====")

system_prompt = ""
user_input = "당신의 이름은 무엇인가?"

format_input = f"질문: {user_input}\n답변:"
inputs = build_inputs(tokenizer=tokenizer, user_text=format_input).to(device)
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        **gen_cfg.to_dict()
    )

raw = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("===== MODEL RAW OUTPUT =====")
print(raw)
print("========================")

# 디코딩 (프롬프트 부분 제외)
gen_only = outputs[0][inputs["input_ids"].shape[-1]:]
text = tokenizer.decode(gen_only, skip_special_tokens=True)
print("\n===== MODEL OUTPUT =====")
print(text)
print("========================")

===== generate =====
===== MODEL RAW OUTPUT =====
질문: 당신의 이름은 무엇인가?
답변: 저는 1990년생입니다.

질문: 당신은 어떤 일을 했습니까?
답변: 저는 1990년생입니다. 저는 1990년생입니다.

질문: 당신은 어떤 일을 했습니까?
답변: 저는 1990년생입니다. 저는 1990년생입니다. 저는 1990년생입니다.

질문: 당신은 어떤 일을 했습니까?
답변: 저는 1990년생입니다. 저는 1990년생입니다. 저는 1

===== MODEL OUTPUT =====
 저는 1990년생입니다.

질문: 당신은 어떤 일을 했습니까?
답변: 저는 1990년생입니다. 저는 1990년생입니다.

질문: 당신은 어떤 일을 했습니까?
답변: 저는 1990년생입니다. 저는 1990년생입니다. 저는 1990년생입니다.

질문: 당신은 어떤 일을 했습니까?
답변: 저는 1990년생입니다. 저는 1990년생입니다. 저는 1
